In [1]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests



def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

#save_sp500_tickers()


def get_data_from_yahoo(reload_sp500=False):
    
    
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle","rb") as f:
            tickers = pickle.load(f)
    
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2000, 1, 1)
    end = dt.datetime(2016, 12, 31)
    
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, "yahoo", start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

#get_data_from_yahoo()

def compile_data():
    with open('sp500tickers.pickle', 'rb') as f:
        tickers = pickle.load(f)
    
    main_df = pd.DataFrame()
    
    for count, ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)
        
        df.rename(columns={'Adj Close':ticker}, inplace=True)
        df.drop(['Open','High','Low','Close','Volume'],1,inplace=True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')
        
        if count % 10 == 0:
            print(count)
        print(main_df.head())
        main_df.to_csv('sp500_joined_closes.csv')
compile_data()
        
            

0
                  MMM
Date                 
2000-01-03  31.333866
2000-01-04  30.088811
2000-01-05  30.960349
2000-01-06  33.450458
2000-01-07  34.114487
                  MMM       ABT
Date                           
2000-01-03  31.333866  9.517434
2000-01-04  30.088811  9.245507
2000-01-05  30.960349  9.228512
2000-01-06  33.450458  9.551425
2000-01-07  34.114487  9.653397
                  MMM       ABT  ABBV
Date                                 
2000-01-03  31.333866  9.517434   NaN
2000-01-04  30.088811  9.245507   NaN
2000-01-05  30.960349  9.228512   NaN
2000-01-06  33.450458  9.551425   NaN
2000-01-07  34.114487  9.653397   NaN
                  MMM       ABT  ABBV  ACN
Date                                      
2000-01-03  31.333866  9.517434   NaN  NaN
2000-01-04  30.088811  9.245507   NaN  NaN
2000-01-05  30.960349  9.228512   NaN  NaN
2000-01-06  33.450458  9.551425   NaN  NaN
2000-01-07  34.114487  9.653397   NaN  NaN
                  MMM       ABT  ABBV  ACN      ATVI


IOError: File stock_dfs/BRK.B.csv does not exist